# Link to public release


https://hsc-release.mtk.nao.ac.jp/datasearch/ 

 # Obtaining the catalogs


Query to obtain the necessary data. Here, we consider the model 'Ephor-AB' as the pz-method:




# After querying the data using the syntax above you should obtain a catalog similar to the one located at: /global/cfs/cdirs/lsst/groups/WL/users/gmarques/hsc_cat_example


# Next, we can verify if the data has the necessary cuts, which are based on  https://arxiv.org/pdf/1705.06745.pdf, Table 4- Appendix 3, and save the catalog in txpipe format.  This is heavily based on https://github.com/LSSTDESC/txpipe-reanalysis/blob/hsc_reanalysis_andrina_dev/scripts/convert_hscdr1photo_h5py.py  
 

In [14]:
import pandas as pd
w12_full =  pd.read_csv('/global/cfs/cdirs/lsst/groups/WL/users/gmarques/hsc_cat_example/wide12h_cat.csv.gz')

In [15]:
w12_full.columns

Index(['# object_id', 'ishape_hsm_regauss_derived_shape_weight',
       'ishape_hsm_regauss_derived_shear_bias_m',
       'ishape_hsm_regauss_derived_shear_bias_c1',
       'ishape_hsm_regauss_derived_shear_bias_c2',
       'ishape_hsm_regauss_derived_sigma_e',
       'ishape_hsm_regauss_derived_rms_e', 'ira', 'idec',
       'ishape_hsm_regauss_e1', 'ishape_hsm_regauss_e2',
       'ishape_hsm_regauss_sigma', 'ishape_hsm_regauss_flags',
       'ishape_hsm_regauss_resolution', 'ishape_hsm_moments_11',
       'ishape_hsm_moments_22', 'ishape_hsm_psfmoments_11',
       'ishape_hsm_psfmoments_22', 'ishape_hsm_psfmoments_12',
       'iclassification_extendedness', 'icmodel_flux', 'icmodel_flux_err',
       'icmodel_mag', 'iblendedness_abs_flux', 'gcmodel_flux',
       'gcmodel_flux_err', 'rcmodel_flux', 'rcmodel_flux_err', 'zcmodel_flux',
       'zcmodel_flux_err', 'ycmodel_flux', 'iflags_pixel_suspect_center',
       'ycmodel_flux_err', 'icmodel_mag_err', 'rcmodel_mag_err', 'rcmodel_mag',
 

In [24]:
import numpy as np
from astropy.table import Table, vstack
import h5py as h5

def apply_cuts(hsc_shearall, field, outputdir): 

    """
    Applies the necessary cuts (based on Table 3. from https://arxiv.org/pdf/1705.06745.pdf) to the full HSC catalog and writes the resulting catalog in TxPipe format.
    Parameters
    ----------
    hsc_shearall: table
        Input shape catalog 

    field: string
        name of the HSC field

    outputdir: string
        Output path to write the catalog

    Returns
    -------
    catalog: table
        Output shape catalog with columns relevant for TxPipe
    """

    
    ngal = len(hsc_shearall)
    print(f'Read {ngal} sources from field: {field}')

    # Basic flag cuts
    unique_det = (hsc_shearall['idetect_is_primary'] == True)
    deblend = (hsc_shearall['ideblend_skipped'] == False)
    centroid = (hsc_shearall['iflags_badcentroid'] == False)
    centr_sdss = (hsc_shearall['icentroid_sdss_flags'] == False)
    edge = (hsc_shearall['iflags_pixel_edge'] == False)
    pix_inter = (hsc_shearall['iflags_pixel_interpolated_center'] == False)
    pix_cen = (hsc_shearall['iflags_pixel_saturated_center'] == False)
    susp = (hsc_shearall['iflags_pixel_suspect_center'] == False)
    clip=  (hsc_shearall['iflags_pixel_clipped_any'] == False)
    
    
    regauss_flag = (hsc_shearall['ishape_hsm_regauss_flags']==False)
    regaus_sigma_nan = (hsc_shearall['ishape_hsm_regauss_sigma'] !=np.nan)
    extendedness = (hsc_shearall['iclassification_extendedness']!=0)
    
    #Cuts on object properties
    
    flux_cmodel = (hsc_shearall['icmodel_flux']/hsc_shearall['icmodel_flux_err']>= 10)
    regauss_resolution = (hsc_shearall['ishape_hsm_regauss_resolution']>=0.3)
    regauss_e = ((hsc_shearall['ishape_hsm_regauss_e1']**2+hsc_shearall['ishape_hsm_regauss_e2']**2)**(1/2)<2)
    regauss_sigma_cut1 = (hsc_shearall['ishape_hsm_regauss_sigma']>=0)
    regauss_sigma_cut2 = (hsc_shearall['ishape_hsm_regauss_sigma']<=0.4)
    imag_cut = (hsc_shearall['icmodel_mag']-hsc_shearall['a_i']<=24.5)
    blendedness_abs_flux = (hsc_shearall['iblendedness_abs_flux']< 10**(-0.375))

    null = (hsc_shearall['ishape_hsm_regauss_e1'] ==0 ) &(hsc_shearall['ishape_hsm_regauss_e2'] ==0)

    all_cuts = (unique_det) & (centroid) & (deblend)& (centr_sdss)& (edge)& (pix_inter)& (pix_cen)& (susp)& (clip)& (extendedness) & (regaus_sigma_nan) & (extendedness) & (regauss_flag) & (flux_cmodel) & (regauss_resolution) & (regauss_e) & (regauss_sigma_cut2) & (regauss_sigma_cut1) & (imag_cut) & (blendedness_abs_flux) 
 
    hsc_shearall = hsc_shearall[all_cuts] 
    #Sorting by ID
    print('sorting data')
    hsc_shearall.sort_values('# object_id')  
    print('sorted data')
    
    print('getting columns')
    
    dec        = hsc_shearall['idec']              
    T     = hsc_shearall['ishape_hsm_moments_11']+hsc_shearall['ishape_hsm_moments_22']                   
    flags = null   
    g1 = hsc_shearall['ishape_hsm_regauss_e1']
    g2 = hsc_shearall['ishape_hsm_regauss_e2']
    mag_err_i    = hsc_shearall['icmodel_mag_err'] 
    mag_err_r    = hsc_shearall['rcmodel_mag_err']
    mag_i    = hsc_shearall['icmodel_mag']     
    mag_r    = hsc_shearall['rcmodel_mag'] 
    psf_T_mean = hsc_shearall['ishape_hsm_psfmoments_11']+ hsc_shearall['ishape_hsm_psfmoments_22']
    # Conversion of moments to e1, e2
    Ixx = hsc_shearall['ishape_hsm_psfmoments_11']
    Iyy = hsc_shearall['ishape_hsm_psfmoments_22']
    Ixy = hsc_shearall['ishape_hsm_psfmoments_12']
    T = Ixx + Iyy
    e1 = (Ixx - Iyy) / T
    e2 = 2*Ixy / T
    psf_g1     = e1      
    psf_g2     = e2      
    s2n        = 1.086/hsc_shearall['icmodel_mag_err']        
    mean_z          = hsc_shearall['photoz_best'] 
    objectId        = hsc_shearall['# object_id']    
    ra              = hsc_shearall['ira']
    snr_i           = 1.086/mag_err_i
    snr_r           = 1.086/mag_err_r
    weight = hsc_shearall['ishape_hsm_regauss_derived_shape_weight']
    m = hsc_shearall['ishape_hsm_regauss_derived_shear_bias_m']
    c1 = hsc_shearall['ishape_hsm_regauss_derived_shear_bias_c1']
    c2 = hsc_shearall['ishape_hsm_regauss_derived_shear_bias_c2']
    sigma_e = hsc_shearall['ishape_hsm_regauss_derived_rms_e']
    flag = hsc_shearall['wl_fulldepth_fullcolor']
    print('loaded columns')
 
    
    #Saving the data as h5file
    data = [dec, T, flags, g1, g2, mag_err_i, mag_err_r, mag_i, mag_r, psf_T_mean, psf_g1, psf_g2, s2n, mean_z, objectId, ra, snr_i, snr_r, weight, m, c1, c2, sigma_e, weight, mean_z, flag]
    
    dnames = ['dec', 'T', 'flags', 'g1', 'g2', 'mag_err_i', 'mag_err_r', 'mag_i', 'mag_r', 'psf_T_mean', 'psf_g1', 'psf_g2', 's2n', 'mean_z', 'objectId', 'ra','snr_i', 'snr_r', 'lensfit_weight', 'm', 'c1', 'c2', 'sigma_e', 'weight', 'redshift_true', 'wl_fulldepth_fullcolor']
    
    print('saving file, ',outputdir + f'shear_catalog_hsc_{field}_nonmetacal.h5')
    
    f = h5.File(outputdir + f'shear_catalog_hsc_{field}_nonmetacal.h5', 'w')
    g = f.create_group('shear')
    for i in range(len(data)):
        g.create_dataset(dnames[i], data=data[i], dtype=data[i].dtype)
    metadata = {'catalog_type':'hsc'}
    g.attrs.update(metadata)
    # write in a group for information on the catalog type 
    f.close()
    
    return data

In [25]:
 
cat_w12_txpipe = apply_cuts(w12_full, 'w12',  '/home/gabriela/Documents/Pesquisa/txpipe_xcross_working/data/hsc/input_cat/')

Read 1192946 sources from field: w12
sorting data
sorted data
getting columns
loaded columns
saving file,  /home/gabriela/Documents/Pesquisa/txpipe_xcross_working/data/hsc/input_catshear_catalog_hsc_w12_nonmetacal.h5
